In [ ]:
# Import libraries

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
from typing import Tuple, Dict, List
# import pprint as pp
import csv
from typing import Tuple, Dict, List
import time
from azureml.core import Workspace

def parse_date(v):
    try:
        return datetime.strptime(v, "%Y-%m-%d %H:%m:%S")
    except:
        # apply whatever remedies you deem appropriate
        pass
    return v

def dict_to_csv(fname: str, dirname: str, _Occs_tot: Dict[tuple, int]):
    with open(dirname+fname+".csv", mode="w", newline='') as f:
        w = csv.writer(f, delimiter=',', quotechar='"',
                       quoting=csv.QUOTE_MINIMAL)

        w.writerow(["Date", "Count"])
        for k, v in tqdm(_Occs_tot.items(), desc=fname):
            w.writerow([k, v])

# Connect to Azure Blob Storage to get data
ws = Workspace.from_config()

datastore = ws.get_default_datastore()
datastore.download("../Data/", prefix="CSV/")

# Data import
cols = ['DateIn', 'DateOut', 'GroCode']
path_to_data = "../Data/CSV/extraction.csv"
data_sample = pd.read_csv(path_to_data, usecols=cols, date_parser=lambda x: parse_date(x), parse_dates=['DateOut', 'DateIn'], encoding="UTF-16 LE", sep=';', quotechar='"', error_bad_lines=False)
print("loaded", len(data_sample))

# Data before 2000 are not valuable, in any, remove cars returned before 2000
data_sample.drop(data_sample[data_sample.DateIn < datetime(2000, 1, 1)].index, inplace=True)
print("dropped datein before 2000-01-01", len(data_sample))

# Drop missing data
data_sample.dropna(inplace=True)

print("Filtered data:", len(data_sample))

mindate = data_sample['DateOut'].min()
maxdate = data_sample['DateIn'].max()

Occs_tot = {}

for _, row in data_sample.iterrows():
    if (row.DateOut >= mindate or row.DateOut < maxdate) and (row.DateOut < mindate or row.DateIn > mindate):
        tdelta = row.DateIn - row.DateOut
        group_code = row.GroCode

        if tdelta.days == 0:
            current_day = row.DateOut
            if current_day < maxdate and current_day >= mindate: 
                k = (group_code, current_day.strftime("%Y-%m-%d"))
                Occs_tot[k] = Occs_tot.setdefault(k, 0) + 1
            
        if tdelta.days > 0:
            for i in range(0, int(tdelta.days), 1):
                current_day = row.DateOut + timedelta(days=i)
                if current_day < maxdate and current_day >= mindate: 
                    k = (group_code, current_day.strftime("%Y-%m-%d"))
                    Occs_tot[k] = Occs_tot.setdefault(k, 0) + 1
    
# calculate occurrencies
occs = {}
for k in Occs_tot.keys():
    j = k[1]
    occs[j] = occs.setdefault(j, 0) + 1   
                   
# output csv
path_to_sink = "../Data/occurrencies/aggregated/"
output_file_name ="occs_tot"
dict_to_csv(output_file_name, path_to_sink, occs)